<a href="https://colab.research.google.com/github/dtabuena/Workshop/blob/main/Py_trodes/read_XYpos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re
import struct
import numpy as np

def extract_position_time(file_dir, filename):
    format_dict = {'uint32': 'I', 'uint16': 'H'}

    os.chdir(file_dir)

    with open(filename, mode='rb') as file:
        filecontent = file.read()

    header = str(filecontent[:200])
    header = header.split('<End settings>')[0]
    field_section = header.split('Fields: ')[-1]
    fields = re.findall(r'<.*?>', field_section)
    units = [f.split(' ')[-1][:-1] for f in fields]
    field_names = [f.split(' ')[0][1:] for f in fields]
    record_format = ''.join([format_dict[u] for u in units])

    header_len = len(header + '<End settings>') - header.count('\\n') - 1
    record_format = ''.join([format_dict[u] for u in units])
    record_size = struct.calcsize(record_format)
    records = []

    with open(filename, 'rb') as file:
        file.seek(header_len)
        while chunk := file.read(record_size):
            if len(chunk) == record_size:
                record = struct.unpack(record_format, chunk)
                records.append(record)

    records = np.stack(records)
    time_data = records[:, 0]
    position_data = records[:, 1:]

    return time_data, position_data

IHHHH


['<time uint32>',
 '<xloc uint16>',
 '<yloc uint16>',
 '<xloc2 uint16>',
 '<yloc2 uint16>']

array([[951221,    269,    385,      0,      0],
       [953087,    268,    385,      0,      0],
       [953087,    267,    387,      0,      0],
       [954952,    267,    388,      0,      0],
       [955884,    268,    387,      0,      0],
       [957749,    269,    387,      0,      0],
       [958682,    269,    388,      0,      0],
       [959615,    270,    388,      0,      0],
       [960547,    269,    388,      0,      0],
       [962412,    269,    389,      0,      0]])